## Advanced regression

Now that you have two new regression methods at your fingertips, it's time to give them a spin. In fact, for this challenge, let's put them together! Pick a dataset of your choice with a binary outcome and the potential for at least 15 features. If you're drawing a blank, the crime rates in 2013 dataset has a lot of variables that could be made into a modelable binary outcome.

Engineer your features, then create three models. Each model will be run on a training set and a test-set (or multiple test-sets, if you take a folds approach).

The models should be:

Vanilla logistic regression

Ridge logistic regression

Lasso logistic regression

If you're stuck on how to begin combining your two new modeling skills, here's a hint: the SKlearn LogisticRegression method has a "penalty" argument that takes either 'l1' or 'l2' as a value.

In your report, evaluate all three models and decide on your best. Be clear about the decisions you made that led to these models (feature selection, regularization parameter selection, model evaluation criteria) and why you think that particular model is the best of the three. Also reflect on the strengths and limitations of regression as a modeling approach. Were there things you couldn't do but you wish you could have done?

In [1]:
import warnings
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import math
import seaborn as sns
import sklearn
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
%matplotlib inline
sns.set_style('white')

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

warnings.filterwarnings("ignore", category=FutureWarning)

The annual County Health Rankings data (http://www.countyhealthrankings.org/explore-health-rankings) provides health and demographic data for every county in the US. It combines both quantitative data (e.g. life expectancy) and qualitative data (e.g. physical distress frequency) to give a snapshot of each county's health profile. This study will ignore most demographic data and focus on more medically-related categories. While demographics no doubt play a role in health, their impact is more difficult to interpret.

In [2]:
full_data = pd.read_csv('../data/chr/County_Health_Rankings.csv',index_col=0)

In [3]:
full_data.head()

,State,County,Life Expectancy,Age-Adjusted Mortality,Child Mortality Rate,Infant Mortality Rate,% Frequent Physical Distress,% Frequent Mental Distress,% Diabetic,HIV Prevalence Rate,% Food Insecure,% Limited Food Access,Drug Overdose Mortality Rate,Motor Vehicle Mortality Rate,% Insufficient Sleep,% Uninsured Adults,% Uninsured Children,% Disconnected Youth,Household Income,% Free or Reduced Lunch,Homicide Rate,Firearm Fatalities Rate,% Homeowners,% Severe Housing Cost Burden,Population,% < 18,% 65 and over,% African American,% American Indian/Alaskan Native,% Asian,% Native Hawaiian/Other Pacific Islander,% Hispanic,% Non-Hispanic White,% Not Proficient in English,% Female,% Rural,Premature Death Years,% Fair/Poor Health,Physically Unhealthy Days,Mentally Unhealthy Days,% Low Birth Wt,% Smokers,% Obese,Food Environment Index,% Physically Inactive,% With Access to Exercise,% Excessive Drinking,% Alcohol-Impaired Deaths,Chlamydia Rate,Teen Birth Rate,% Uninsured,PCP Ratio,Dentist Ratio,MHP Ratio,Preventable Hosp. Rate,% Mammograph Screened,% Flu Vaccinated,HS Graduation Rate,% Some College,% Unemployed,% Children in Poverty,80/20 Income Ratio,% Single-Parent Households,Social Association Rate,Violent Crime Rate,Injury Death Rate,Average Daily Particulate Matter 2.5,Presence of drinking water violation,% Severe Housing Problems,% Drive Alone,% Long Commute - Drives Alone
FIPS,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1001,Alabama,Autauga,76.300,439.000,53.000,8.000,13,13,14,226.000,13,12.000,10.000,20.000,36,11.000,2.000,8.000,58343.000,48.000,5.000,18.000,73,13.000,55504,23.900,15.100,19.300,0.500,1.300,0.100,2.900,74.500,1,51.300,42.000,0.159,18,4.200,4.300,8.000,19,38,7.200,31,69.000,17,29.000,341.200,27.000,9.000,2409,3084,6167,6599.000,44.000,41.000,90.000,61,3.900,19.000,4.600,25.000,12.600,272.000,74.000,11.700,No,15,86,38
1003,Alabama,Baldwin,78.600,348.000,47.000,6.000,13,13,11,164.000,12,5.000,16.000,15.000,33,14.000,3.000,8.000,56607.000,45.000,3.000,14.000,73,13.000,212628,21.800,19.900,9.000,0.800,1.200,0.100,4.600,83.000,0,51.500,42.300,0.034,18,4.100,4.200,8.000,17,31,8.000,24,72.000,17,32.000,338.800,30.000,11.000,1372,2006,1096,3833.000,45.000,45.000,86.000,66,4.000,15.000,4.500,25.000,10.700,204.000,69.000,10.300,Yes,14,85,41
1005,Alabama,Barbour,75.800,470.000,77.000,nan,16,15,18,436.000,23,11.000,nan,21.000,39,17.000,3.000,12.000,32490.000,74.000,7.000,15.000,63,14.000,25270,20.800,18.800,47.900,0.700,0.500,0.200,4.200,46.000,1,47.200,67.800,0.379,26,5.100,4.600,11.000,22,44,5.600,28,54.000,13,30.000,557.900,45.000,13.000,2597,2808,12635,4736.000,46.000,37.000,81.000,37,5.900,50.000,5.800,57.000,8.500,414.000,73.000,11.500,No,15,83,34
1007,Alabama,Bibb,73.900,564.000,112.000,15.000,13,13,15,192.000,16,3.000,22.000,25.000,38,12.000,3.000,nan,45795.000,65.000,8.000,21.000,75,9.000,22668,20.600,16.000,21.500,0.400,0.200,0.100,2.600,74.300,0,46.500,68.400,0.520,20,4.400,4.300,11.000,20,38,7.600,35,16.000,16,27.000,302.100,45.000,10.000,1742,3778,11334,5998.000,44.000,39.000,84.000,48,4.400,27.000,4.300,30.000,10.200,89.000,100.000,11.200,No,11,86,49
1009,Alabama,Blount,74.600,502.000,76.000,6.000,14,14,14,95.000,11,3.000,25.000,26.000,36,16.000,3.000,15.000,48253.000,53.000,7.000,20.000,79,8.000,58013,23.300,17.800,1.500,0.600,0.300,0.100,9.600,86.900,2,50.700,90.000,0.188,21,4.500,4.700,8.000,20,34,8.500,29,23.000,15,22.000,114.300,36.000,12.000,4439,4834,9669,4162.000,36.000,38.000,93.000,54,4.000,19.000,4.100,30.000,9.000,483.000,105.000,11.700,No,10,87,60


In [4]:
# Select medically-related fields
data = pd.DataFrame(index=full_data.index)
data['Life Expectancy'] = full_data['Life Expectancy']
data['Physical Distress Pct'] = full_data['% Frequent Physical Distress']
data['Mental Distress Pct'] = full_data['% Frequent Mental Distress']
data['Diabetic Pct'] = full_data['% Diabetic']
data['HIV Rate'] = full_data['HIV Prevalence Rate']
data['Food Insecure Pct'] = full_data['% Food Insecure']
data['Insufficient Sleep Pct'] = full_data['% Insufficient Sleep']
data['Houshold Income'] = full_data['Household Income']
data['Youth Pct'] = full_data['% < 18']
data['Elderly Pct'] = full_data['% 65 and over']
data['Female Pct'] = full_data['% Female']
data['Poor Health Pct'] = full_data['% Fair/Poor Health']
data['Physically Unhealthy Days'] = full_data['Physically Unhealthy Days']
data['Mentally Unhealthy Days'] = full_data['Mentally Unhealthy Days']
data['Low Birth Weight Pct'] = full_data['% Low Birth Wt']
data['Smoker Pct'] = full_data['% Smokers']
data['Obesity Pct'] = full_data['% Obese']
data['Inactive Pct'] = full_data['% Physically Inactive']
data['Exercise Availability Pct'] = full_data['% With Access to Exercise']
data['Excess Drinker Pct'] = full_data['% Excessive Drinking']
data['Chlamydia Rate'] = full_data['Chlamydia Rate']
data['Teen Birth Rate'] = full_data['Teen Birth Rate']
data['PCP Ratio'] = full_data['PCP Ratio']
data['Dentist Ratio'] = full_data['Dentist Ratio']
data['MHP Ratio'] = full_data['MHP Ratio']
data['Mammograph Pct'] = full_data.iloc[:,[-16]] #full_data['% Mammograph Screened']
data['Flu Vaccinated Pct'] = full_data['% Flu Vaccinated']

In [5]:
# Check whether data cleaning is needed
data.dtypes

Life Expectancy              float64
Physical Distress Pct          int64
Mental Distress Pct            int64
Diabetic Pct                   int64
HIV Rate                     float64
Food Insecure Pct              int64
Insufficient Sleep Pct         int64
Houshold Income              float64
Youth Pct                    float64
Elderly Pct                  float64
Female Pct                   float64
Poor Health Pct                int64
Physically Unhealthy Days    float64
Mentally Unhealthy Days      float64
Low Birth Weight Pct         float64
Smoker Pct                     int64
Obesity Pct                    int64
Inactive Pct                   int64
Exercise Availability Pct    float64
Excess Drinker Pct             int64
Chlamydia Rate               float64
Teen Birth Rate              float64
PCP Ratio                     object
Dentist Ratio                 object
MHP Ratio                     object
Mammograph Pct               float64
Flu Vaccinated Pct           float64
d

In [6]:
# Handle non-numeric columns
data['PCP Ratio'] = pd.to_numeric(data['PCP Ratio'], errors='coerce')
data['Dentist Ratio'] = pd.to_numeric(data['Dentist Ratio'], errors='coerce')
data['MHP Ratio'] = pd.to_numeric(data['MHP Ratio'], errors='coerce')

There is a considerable amount of co-linearity with some high correlations but try to allow the model to handle them.

In [7]:
data.corr()

,Life Expectancy,Physical Distress Pct,Mental Distress Pct,Diabetic Pct,HIV Rate,Food Insecure Pct,Insufficient Sleep Pct,Houshold Income,Youth Pct,Elderly Pct,Female Pct,Poor Health Pct,Physically Unhealthy Days,Mentally Unhealthy Days,Low Birth Weight Pct,Smoker Pct,Obesity Pct,Inactive Pct,Exercise Availability Pct,Excess Drinker Pct,Chlamydia Rate,Teen Birth Rate,PCP Ratio,Dentist Ratio,MHP Ratio,Mammograph Pct,Flu Vaccinated Pct
Life Expectancy,1.000,-0.667,-0.689,-0.642,-0.106,-0.598,-0.526,0.625,-0.159,0.032,-0.122,-0.637,-0.668,-0.635,-0.477,-0.698,-0.545,-0.628,0.360,0.539,-0.326,-0.675,-0.179,-0.258,-0.134,0.346,0.224
Physical Distress Pct,-0.667,1.000,0.938,0.619,0.141,0.684,0.660,-0.687,0.085,-0.075,0.055,0.920,0.974,0.870,0.512,0.806,0.415,0.540,-0.307,-0.644,0.351,0.680,0.202,0.254,0.100,-0.433,-0.261
Mental Distress Pct,-0.689,0.938,1.000,0.660,0.118,0.700,0.673,-0.658,0.027,-0.035,0.148,0.837,0.942,0.941,0.501,0.808,0.432,0.541,-0.284,-0.651,0.335,0.599,0.174,0.229,0.066,-0.373,-0.190
Diabetic Pct,-0.642,0.619,0.660,1.000,0.120,0.540,0.601,-0.561,-0.074,0.221,0.208,0.597,0.637,0.652,0.478,0.615,0.660,0.748,-0.399,-0.653,0.160,0.472,0.241,0.312,0.226,-0.167,-0.137
HIV Rate,-0.106,0.141,0.118,0.120,1.000,0.365,0.346,-0.071,-0.084,-0.164,-0.055,0.236,0.089,0.058,0.416,0.083,0.022,0.087,0.006,-0.098,0.519,0.120,0.020,-0.012,-0.008,-0.044,-0.102
Food Insecure Pct,-0.598,0.684,0.700,0.540,0.365,1.000,0.522,-0.605,-0.029,-0.060,0.097,0.676,0.648,0.606,0.612,0.584,0.377,0.448,-0.259,-0.536,0.528,0.503,0.129,0.156,0.061,-0.297,-0.239
Insufficient Sleep Pct,-0.526,0.660,0.673,0.601,0.346,0.522,1.000,-0.314,-0.011,-0.255,0.103,0.653,0.682,0.689,0.538,0.639,0.432,0.478,-0.119,-0.450,0.329,0.391,0.151,0.198,0.058,-0.202,0.022
Houshold Income,0.625,-0.687,-0.658,-0.561,-0.071,-0.605,-0.314,1.000,0.086,-0.291,0.017,-0.671,-0.652,-0.559,-0.407,-0.596,-0.456,-0.576,0.428,0.534,-0.194,-0.627,-0.199,-0.262,-0.174,0.304,0.392
Youth Pct,-0.159,0.085,0.027,-0.074,-0.084,-0.029,-0.011,0.086,1.000,-0.587,0.180,0.169,0.040,-0.050,-0.080,0.085,0.180,0.066,-0.076,-0.060,0.243,0.345,0.041,-0.004,0.028,-0.223,-0.094
Elderly Pct,0.032,-0.075,-0.035,0.221,-0.164,-0.060,-0.255,-0.291,-0.587,1.000,0.074,-0.145,-0.060,-0.021,-0.057,-0.141,-0.059,0.119,-0.175,-0.161,-0.369,-0.085,0.012,0.068,0.088,0.129,-0.194


A simple measure of health is life expectancy. As a classifer, counties can be divided into those above and below average life expectancy (not population-adjusted).

In [8]:
# Use above/below average life expectancy as classifier
X = data.dropna()
Y = (X['Life Expectancy'] > X['Life Expectancy'].mean())*1
X = X.drop('Life Expectancy', 1)

In [9]:
# Create training and test sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

Vanilla Logistic Regression:

In [10]:
log_reg = linear_model.LogisticRegression(C=1e40) # use arbitrarily high C to negate regularization
log_fit = log_reg.fit(X_train, Y_train)

cv_score = cross_val_score(log_fit, X, Y, cv=10)
print("Vanilla Logistic Regression Accuracy: %0.5f (+/- %0.5f)" % (cv_score.mean(), cv_score.std() * 2))

Vanilla Logistic Regression Accuracy: 0.85173 (+/- 0.11117)


Ridge Logistic Regression:

In [11]:
ridge_reg = linear_model.LogisticRegressionCV(cv=10,solver='liblinear',penalty='l2',max_iter=1000).fit(X_train,Y_train)
ridge_reg.score(X_test,Y_test)
cv_score = cross_val_score(ridge_reg,X_test,Y_test)
print("Ridge Logistic Regression Accuracy: %0.5f (+/- %0.5f)" % (cv_score.mean(), cv_score.std() * 2))

Ridge Logistic Regression Accuracy: 0.84213 (+/- 0.02063)


Lasso Logistic Regression:

In [12]:
lasso_reg = linear_model.LogisticRegressionCV(cv=10,solver='liblinear',penalty='l1',max_iter=1000).fit(X_train,Y_train)
lasso_reg.score(X_test,Y_test)
cv_score = cross_val_score(lasso_reg,X_test,Y_test)
print("Lasso Logistic Regression Accuracy: %0.5f (+/- %0.5f)" % (cv_score.mean(), cv_score.std() * 2))

Lasso Logistic Regression Accuracy: 0.82897 (+/- 0.02057)


Conclusions:

Of the three regressions, the vanilla logistic regression yields the best accuracy but shows considerably more variation in cross validation scores. Its accuracy is about 0.010 better than the ridge regression and about 0.023 better than the lasso regression. Variation across the ridge and lasso regression cross validation scores are similar so of the three, the ridge regression has the best combination of accuracy and robustness.